# 4 Equity Index Momentum Strategy (MOM)

(a) 
Construct the return to a long-short momentum strategy portfolio. To that effect every month sort currency hedged stock indexes based on their 1-month lagged 11-month return (that is in month trank stocks based on their t− 12 to t− 1 cumulative return). Call Rankit the corresponding rank of index i at time t (e.g., Rankit = 2 if country i has the second lowest Currency-hedged return over the last year.)
Then compute monthly returns to a portfolio that invests in index i the weight
  wit = Z(Ranki−(N + 1)/2 )
for all i= 1,...,N, and where N is the total number of stock indexes traded and Z is a factor that insures that the the sum of the long positions is +$1 and the sum of the short positons is−$1.

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Load the dataset and set 'date' as the index
df = pd.read_csv("merged_all_data.csv", parse_dates=['date'])
df.set_index('date', inplace=True)

# Define the list of countries and prefix for currency-hedged returns
countries = ['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'UNITED KINGDOM', 'JAPAN']
prefix = 'mportretx_'  # monthly portfolio returns, currency-hedged

# Create a DataFrame to store momentum signals
momentum = pd.DataFrame(index=df.index)

# Compute past 11-month cumulative log-returns for each country
for country in countries:
    col = prefix + country
    log_ret = np.log(1 + df[col])
    momentum[country] = log_ret.rolling(window=11, min_periods=11).sum().shift(1)  # t−12 to t−1 returns

# Rank countries each month based on their momentum
ranks = momentum.rank(axis=1, method='first')

# Compute ranking weights for the long-short portfolio
N = len(countries)
Z = 2 / (N * (N - 1))
weights = (ranks - (N + 1) / 2) * Z  # assigns positive weights to winners, negative to losers

# Extract return data and rename columns
returns = df[[prefix + c for c in countries]].copy()
returns.columns = countries

# Compute the return of the momentum strategy
MOM_return = (weights * returns).sum(axis=1)

(b) Compute and compare the mean, standard deviation, and Sharpe ratios of the long
and short legs of the strategy as well as of the strategy itself. Test if the strategy has
an average return that is statistically significantly different from zero.

In [8]:
# Define long and short positions based on ranks
# Long: top 3 countries by momentum
# Short: bottom 3 countries by momentum
long_mask = ranks.apply(lambda row: row >= (N - 3 + 1), axis=1)
short_mask = ranks.apply(lambda row: row <= 3, axis=1)

# Compute average return of long and short legs
long_returns = (returns * long_mask).sum(axis=1) / long_mask.sum(axis=1)
short_returns = (returns * short_mask).sum(axis=1) / short_mask.sum(axis=1)

# Momentum strategy return (already computed before)
strategy_returns = MOM_return

import pandas as pd
import numpy as np
from scipy import stats

# Create a DataFrame to summarize mean, standard deviation, and Sharpe ratio
results = pd.DataFrame({
    "Mean": [long_returns.mean(), short_returns.mean(), strategy_returns.mean()],
    "Std": [long_returns.std(), short_returns.std(), strategy_returns.std()],
    "Sharpe": [
        long_returns.mean() / long_returns.std(),
        short_returns.mean() / short_returns.std(),
        strategy_returns.mean() / strategy_returns.std()
    ]
}, index=["Long", "Short", "Strategy"])

# Perform t-test to check if strategy return is significantly different from 0
t_stat, p_value = stats.ttest_1samp(strategy_returns.dropna(), popmean=0)
results["t-stat"] = [np.nan, np.nan, t_stat]
results["p-value"] = [np.nan, np.nan, p_value]

# Display result rounded to 4 decimal places
print(results.round(4))

            Mean     Std  Sharpe  t-stat  p-value
Long      0.0034  0.0382  0.0892     NaN      NaN
Short     0.0053  0.0384  0.1372     NaN      NaN
Strategy -0.0007  0.0072 -0.0965 -1.6026   0.1102
